In [9]:
import os
import re
import pandas as pd

# Basisverzeichnis für die Verarbeitung
BASE_DIR = "data/nist"

# Dateinamensmuster für die verschiedenen Messungen
MASSIF_FILES = {
    "keygen_heap": "massif.out.keygen.heap",
    "keygen_stack": "massif.out.keygen.stack",
    "enc_heap": "massif.out.enc.heap",
    "enc_stack": "massif.out.enc.stack",
    "dec_heap": "massif.out.dec.heap",
    "dec_stack": "massif.out.dec.stack",
}

def extract_model_name(directory_name):
    """Extrahiert den Modelnamen aus dem Verzeichnisnamen."""
    match = re.match(r"testresult_([^_]+)", directory_name)
    return match.group(1) if match else directory_name

def parse_massif_file(file_path):
    """Liest eine Massif-Datei aus und gibt die maximale Speicherbelegung zurück."""
    max_heap = 0
    max_stack = 0
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                
                if file_path.endswith(".heap"):
                    if line.startswith("mem_heap_B="):
                        heap_value = int(line.split("=")[1].strip())
                        current_heap = heap_value
                    elif line.startswith("mem_heap_extra_B="):
                        extra_value = int(line.split("=")[1].strip())
                        total_heap = current_heap + extra_value
                        if total_heap > max_heap:
                            max_heap = total_heap
                            #print (file_path.split("massif.out.")[1].strip(), max_heap)
                
                elif file_path.endswith(".stack"):
                    if line.startswith("mem_stacks_B="):
                        stack_value = int(line.split("=")[1].strip())
                        if stack_value > max_stack:
                            max_stack = stack_value
    except Exception as e:
        print(f"Fehler beim Lesen der Datei {file_path}: {e}")
    
    return max_heap, max_stack

# Hauptfunktion zum Sammeln der Daten
def main():
    all_records = []
    
    for model_dir in os.listdir(BASE_DIR):
        model_path = os.path.join(BASE_DIR, model_dir)
        if os.path.isdir(model_path):
            model_name = extract_model_name(model_dir)
            
            for algo_dir in os.listdir(model_path):
               
                algo_path = os.path.join(model_path, algo_dir)
                if os.path.isdir(algo_path):
                    record = {"Model": model_name, "Algorithmus": algo_dir}
                    
                    for key, filename in MASSIF_FILES.items():
                        file_path = os.path.join(algo_path, filename)
                        if os.path.isfile(file_path):
                            heap, stack = parse_massif_file(file_path)
                            if "heap" in key:
                                record[key] = heap
                            else:
                                record[key] = stack
                        else:
                            record[key] = None
                    
                    all_records.append(record)
    
    # DataFrame erstellen
    df = pd.DataFrame(all_records)
    df = df.sort_values(by=["Algorithmus", "Model"])
    
    # Speichere als CSV
    df.to_csv("output/csv/nist_mem.csv", index=False)
    
    # Anzeige im Jupyter Notebook
    from IPython.display import display
    display(df)
    
    return df

# Starte die Verarbeitung
df = main()

,Model,Algorithmus,keygen_heap,keygen_stack,enc_heap,enc_stack,dec_heap,dec_stack
119,Pi1b,BIG_QUAKE_1,11640832,17524,4780032,15340,4734976,15748
0,Pi1bPlus,BIG_QUAKE_1,11640832,17524,4780032,15340,4734976,15748
236,Pi3b,BIG_QUAKE_1,12484608,18256,5562368,16080,5517312,16512
423,Pi4b,BIG_QUAKE_1,12484608,18256,5562368,16080,5517312,16512
610,PiBanana,BIG_QUAKE_1,15601664,17864,8736768,15200,8392704,15888
...,...,...,...,...,...,...,...,...
590,Pi4b,sphincs-shake256-256s-robust,5365760,9328,5365760,9264,5545984,38208
1015,PiZero,sphincs-shake256-256s-robust,4579328,8928,4579328,8824,4747264,37808
404,Pi3b,sphincs-shake256-256s-simple,5365760,8880,5365760,8880,5545984,36224
591,Pi4b,sphincs-shake256-256s-simple,5365760,8880,5365760,8880,5545984,36288
